In [0]:
import requests

url = "https://opencom.no/dataset/36ceda99-bbc3-4909-bc52-b05a6d634b3f/resource/d1bdc6eb-9b49-4f24-89c2-ab9f5ce2acce/download/parking.json"

try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    dbutils.notebook.exit(f"Feil ved henting av data: {str(e)}")

df_raw = spark.createDataFrame(data)

bronze_path = "/mnt/bronze/staging_parking"

(
    df_raw.write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .save(bronze_path)
)

spark.sql(f"""
CREATE TABLE IF NOT EXISTS default.staging_parking
USING DELTA
LOCATION '{bronze_path}'
""")